# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pymongo.server_api import ServerApi
from pprint import pprint
from atlascompass import mongopass
import pandas as pd
from bson import ObjectId

In [ ]:
# Create an instance of MongoClient
# mongo = MongoClient(port=27017)

## not using the local host due to issues with other services running on the same port.Instead, using the API to the free cloud service.

In [2]:
uri = "mongodb+srv://kuromasadev:{}@clusteraprda.x7kvjsy.mongodb.net/?retryWrites=true&w=majority".format(mongopass)
# Create a new client and connect to the server
mongo = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    mongo.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
# confirm that our new database was created

database_names = mongo.list_database_names()

if "uk_food" in database_names:
    print("The new database was created.")
else:
    print("The new database was not found.")

The new database was created.


In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
collection_names = db.list_collection_names()

if collection_names:
    print("Collections in the new database:")
    for collection_name in collection_names:
        print(collection_name)
else:
    print("No collections found in the new database.")


Collections in the new database:
raw_import
establishments


In [6]:
# review a document in the establishments collection
collection = db['establishments']

documents = collection.find()

document_preview = pd.DataFrame(documents)

document_preview.head(3)

,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,6490fa85916b488c4a3c5485,254719,0,PI/000069980,Refreshment Kiosk,Restaurant/Cafe/Canteen,1,The Bay,St Margarets Bay,Kent,...,http://www.dover.gov.uk/,publicprotection@dover.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '1.387974', 'latitude': '51.1522...",,4587.347175,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
1,6490fa85916b488c4a3c5486,1034540,0,PI/000078691,The Coastguard Inn,Pub/bar/nightclub,7843,The Bay,St Margarets Bay,Kent,...,http://www.dover.gov.uk/,publicprotection@dover.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '1.387974', 'latitude': '51.1522...",,4587.347175,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
2,6490fa85916b488c4a3c5487,254250,0,PI/000066174,The Pines Calyx,Other catering premises,7841,The Pines Garden,Beach Road,St Margarets Bay,...,http://www.dover.gov.uk/,publicprotection@dover.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '1.383298', 'latitude': '51.1481...",,4587.362403,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."


In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [9]:
# Insert the new restaurant into the collection
insert_result = collection.insert_one(new_restaurant)


In [10]:
# Check that the new restaurant was inserted
if insert_result.inserted_id:
    print("New restaurant inserted successfully.")
else:
    print("Failed to insert the new restaurant.")


New restaurant inserted successfully.


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type = collection.find_one(
    {"BusinessType": "Restaurant/Cafe/Canteen"},
    {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
)

In [12]:
print(business_type)

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
object_id = ObjectId("6490fb0cb7b56b965095953d")
document = collection.find_one({"_id": object_id})

In [16]:
pprint(document)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6490fb0cb7b56b965095953d'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [17]:
# Update the new restaurant with the correct BusinessTypeID
insert_fiter_1 = {"name": "Penang Flavours"}
updated_data = {"$set": {"BusinessTypeID": "1"}}
updated_result = collection.update_one(document, updated_data)


In [18]:
object_id = ObjectId("6490fb0cb7b56b965095953d")
check_document = collection.find_one({"_id": object_id})

In [19]:
pprint(check_document)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6490fb0cb7b56b965095953d'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [20]:
# Find how many documents have LocalAuthorityName as "Dover"
count_dover = collection.count_documents({"LocalAuthorityName": "Dover"})
print(f"Number of documents with LocalAuthorityName as 'Dover': {count_dover}")


Number of documents with LocalAuthorityName as 'Dover': 994


In [21]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = collection.delete_many({"LocalAuthorityName": "Dover"})
print(f"Deleted {delete_result.deleted_count} documents with LocalAuthorityName as 'Dover'")

Deleted 994 documents with LocalAuthorityName as 'Dover'


In [22]:
# Check if any remaining documents include Dover
count_remaining = collection.count_documents({"LocalAuthorityName": "Dover"})
if count_remaining == 0:
    print("No documents remaining with LocalAuthorityName as 'Dover'")
else:
    print("There are remaining documents with LocalAuthorityName as 'Dover'")


No documents remaining with LocalAuthorityName as 'Dover'


In [23]:
# Check that other documents remain with 'find_one'
remaining_document = collection.find_one()
remaining_count = collection.count_documents({})

if remaining_document:
    print(f"There are {remaining_count} remaining documents.")
else:
    print("No documents remaining.")


There are 38786 remaining documents.


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [28]:
# Change the data type from String to Decimal for longitude and latitude
update_latlong = collection.update_many({}, [
    {"$set": {"longitude": {"$convert": {"input": "$longitude", "to": "double"}}}},
    {"$set": {"latitude": {"$convert": {"input": "$latitude", "to": "double"}}}}
])

# Check if the update was successful
if update_latlong.modified_count > 0:
    print("Data type updated successfully.")
else:
    print("No changes were made.")

No changes were made.


Use `update_many` to convert `RatingValue` to integer numbers.

In [25]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [26]:
# Change the data type from String to Integer for RatingValue
collection.update_many({"RatingValue": {"$exists": True}}, [{"$set": {"RatingValue": {"$toInt": "$RatingValue"}}}])


In [39]:
document = collection.find_one()

In [40]:
# Check that the coordinates and rating value are now numbers
print("Sample Document:")
pprint(document)

print("Coordinates (Longitude, Latitude):", type(document["geocode"]["longitude"]), type(document["geocode"]["latitude"]))
print("RatingValue:", type(document["RatingValue"]))

Sample Document:
{'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4591.765489457773,
 'FHRSID': 1043695,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6BL',
 'RatingDate': '2018-04-04T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': 5,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6490fa85916b488c4a3c576c'),
 'geocode': {'latitude': '51.083812', 'longitude': '1.195625'},
 'latitude': None,
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
            'rel': 

In [41]:
### alternative approach, as the update.many() was not seemingly working.

from bson.decimal128 import Decimal128
pipeline_latlon = [
    {
        "$addFields": {
            "geocode.longitude": {
                "$toDouble": {
                    "$ifNull": [
                        "$geocode.longitude",
                        Decimal128("0")
                    ]
                }
            },
            "geocode.latitude": {
                "$toDouble": {
                    "$ifNull": [
                        "$geocode.latitude",
                        Decimal128("0")
                    ]
                }
            }
        }
    }
]

update_result = collection.aggregate(pipeline_latlon)

# Check if the update was successful
if update_result.alive:
    print("Data type updated successfully.")
else:
    print("No changes were made.")

Data type updated successfully.


## Shutoff

In [ ]:
mongo.close()